In [1]:
import pickle
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import tree
# from bboxes.dtbb import DecisionTreeBlackBox
# from bboxes.nnbb import NeuralNetworkBlackBox
# from bboxes.rfbb import RandomForestBlackBox
from sklearn.model_selection import GridSearchCV
from new_trepan.trepan_generation import TrePanGenerator
import numpy as np
from tqdm import tqdm
import os
from abstract import *
import torch
import torch.nn as nn

import torch.nn.functional as F


In [2]:



class SimpleNet(nn.Module):
    def __init__(self, hidden_1, dropout, output, input):
        super(SimpleNet, self).__init__()

        self.fc1 = nn.Linear(input, hidden_1)
        self.fc2 = nn.Linear(hidden_1, output)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x


class Net(nn.Module):

    def __init__(self, hidden_1, hidden_2, dropout, output,
                 input
                 ):

        super(Net, self).__init__()
        self.fc1 = nn.Linear(input, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.drop = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_2, output)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = self.out(x)
        return x


class Layer3Net(nn.Module):

    def __init__(self, hidden_1, hidden_2, hidden_3,
                 dropout, output, input
                 ):

        super(Layer3Net, self).__init__()
        self.fc1 = nn.Linear(input, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.drop = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_3, output)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.drop(x)
        x = self.out(x)
        return x


def create_simple_net(hidden_1, dropout, output, input):
    net = SimpleNet(hidden_1, dropout, output, input)
    return net


def create_net(hidden_1=300, hidden_2=100, dropout=0.1, output=2,
               input=85):
    net = Net(hidden_1, hidden_2, dropout, output,
              input)
    return net

def create_layer3net(hidden_layer_1, hidden_layer_2, hidden_layer_3,
                     dropout_rate, n_classes, n_features):
    net = Layer3Net(hidden_layer_1, hidden_layer_2, hidden_layer_3,
                    dropout_rate, n_classes, n_features)
    return net


In [3]:



class SimpleNet(nn.Module):
    def __init__(self, hidden_1, dropout, output, input):
        super(SimpleNet, self).__init__()

        self.fc1 = nn.Linear(input, hidden_1)
        self.fc2 = nn.Linear(hidden_1, output)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x


class Net(nn.Module):

    def __init__(self, hidden_1, hidden_2, dropout, output,
                 input
                 ):

        super(Net, self).__init__()
        self.fc1 = nn.Linear(input, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.drop = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_2, output)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = self.out(x)
        return x


class Layer3Net(nn.Module):

    def __init__(self, hidden_1, hidden_2, hidden_3,
                 dropout, output, input
                 ):

        super(Layer3Net, self).__init__()
        self.fc1 = nn.Linear(input, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.drop = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_3, output)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.drop(x)
        x = self.out(x)
        return x


def create_simple_net(hidden_1, dropout, output, input):
    net = SimpleNet(hidden_1, dropout, output, input)
    return net


def create_net(hidden_1=300, hidden_2=100, dropout=0.1, output=2,
               input=85):
    net = Net(hidden_1, hidden_2, dropout, output,
              input)
    return net

def create_layer3net(hidden_layer_1, hidden_layer_2, hidden_layer_3,
                     dropout_rate, n_classes, n_features):
    net = Layer3Net(hidden_layer_1, hidden_layer_2, hidden_layer_3,
                    dropout_rate, n_classes, n_features)
    return net


In [4]:
bb0 = create_net(input = 39, hidden_1  = 256, hidden_2 = 256, dropout = 0.1, output = 2  )
ckpt_0 = torch.load(open('path of the model', 'rb'))
value_0 = ckpt_0.model
bb0.load_state_dict(value_0)
bb0.eval()

Net(
  (fc1): Linear(in_features=39, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (drop): Dropout(p=0.1, inplace=False)
  (out): Linear(in_features=256, out_features=2, bias=True)
)

In [5]:
def new_score(accuracy, depth, weight_accuracy=1.0, weight_depth=1.0):
    score = (weight_accuracy * accuracy) + (weight_depth * (1 / depth))
    return score

In [6]:
def new_score_sorting(config_list, gs_scores):
    scores = []
    for i, conf in enumerate(config_list):
        depth = conf['max_depth']
        sr = gs_scores[i]
        new_sr = new_score(sr, depth, weight_accuracy=0.2, weight_depth=0.1)
        scores.append((new_sr, sr, depth, conf))
    scores.sort(key=lambda x: x[0], reverse=True)
    """for s in scores:
        print("new_score:{} acc:{} depth:{}".format(round(s[0], 2), s[1], s[2]))"""
    return scores[0][3]

In [7]:
def train_explainer(dataset_name, best_param, tr_set, tr_label, ts_set, ts_label, path_tuple, lss_dpt=False):
    dt = tree.DecisionTreeClassifier(**best_param)
    dt.fit(tr_set, tr_label)
    predictions1 = dt.predict(tr_set)
    report = classification_report(tr_label, predictions1)
    write_report = open("./explainers/{}/{}/{}/best_param{}.txt".format(dataset_name, path_tuple[0], path_tuple[1],
                                                                        "_lssdpt" if lss_dpt else ""), "w")
    write_report.write(str(best_param))
    write_report.close()

    write_report = open("./explainers/{}/{}/{}/tr_fidelity{}.txt".format(dataset_name, path_tuple[0], path_tuple[1],
                                                                         "_lssdpt" if lss_dpt else ""), "w")
    write_report.write(report)
    write_report.close()

    predictions = dt.predict(ts_set)
    report = classification_report(ts_label, predictions)

    write_report = open("./explainers/{}/{}/{}/ts_fidelity{}.txt".format(dataset_name, path_tuple[0], path_tuple[1],
                                                                         "_lssdpt" if lss_dpt else ""), "w")
    write_report.write(report)
    write_report.close()

    # Train fidelity
    fidelity_train = dt.predict(train_set)
    report = classification_report(train_label, fidelity_train)
    write_report = open("./explainers/{}/{}/{}/tr_original{}.txt".format(dataset_name, path_tuple[0], path_tuple[1],
                                                                         "_lssdpt" if lss_dpt else ""), "w")
    write_report.write(report)
    write_report.close()

    # Test fidelity
    fidelity_test = dt.predict(test_set)
    report = classification_report(test_label, fidelity_test)
    write_report = open(
        "./explainers/{}/{}/{}/ts_original{}.txt".format(dataset_name, path_tuple[0], path_tuple[1],
                                                         "_lssdpt" if lss_dpt else ""), "w")
    write_report.write(report)
    write_report.close()

    filename = "./explainers/{}/{}/{}/explainer{}.sav".format(dataset_name, path_tuple[0], path_tuple[1],
                                                              "_lssdpt" if lss_dpt else "")
    pickle.dump(dt, open(filename, 'wb'))
    write_report.close()

In [8]:
def greater_than_85(config_list, gs_scores):
    """
    We select the model having accuracy greater
    than 85 and the minimum number of depth.
    :param config_list:
    :param gs_scores:
    :return:
    """
    coupled = [(gs_scores[i], conf['max_depth'], conf) for i, conf in enumerate(config_list)]
    filtered = list(filter(lambda x: x[0] >= 0.77, coupled))
    filtered.sort(key=lambda x: x[0], reverse=False)
    print(filtered)
    return filtered[0][2]

In [9]:
def train_explainer_regularized(ds_name, train_set, train_label, test_set, test_label, path_tuple, max_depth_l=[8, 9, 10, 11, 12, 13]):
    tree_para = {'criterion': ['gini', 'entropy'], 'max_depth': max_depth_l,
                 'min_samples_split': [5, 10, 15, 25, 30, 50], 'min_samples_leaf': [3, 5, 15, 20, 40, 50],
                 'max_features': [5, 'auto', 'sqrt', 'log2']}
    grid = GridSearchCV(tree.DecisionTreeClassifier(), tree_para, cv=3, n_jobs=12, verbose=10, scoring='accuracy')
    grid.fit(train_set, train_label.ravel())
    configurations_list = grid.cv_results_['params']
    scores_list = grid.cv_results_['mean_test_score']
    # Select the way you want to perform the model selection
    best_param = new_score_sorting(configurations_list, scores_list)
    # best_param = greater_than_85(configurations_list, scores_list)
    # best_param = grid.best_params_
    # normal training of the explainer
    train_explainer(ds_name, best_param, train_set, train_label, test_set, test_label, path_tuple, lss_dpt=True)
    # train_explainer_for_plot(best_param, train_set, train_label, test_set, test_label, max_depth_l)

In [10]:
regularizeds = False

In [11]:
dt_best_hyperparams = [
    {'criterion': 'entropy', 'max_depth': 20, 'max_features': 5, 'min_samples_leaf': 40, 'min_samples_split': 50},
    {'criterion': 'gini', 'max_depth': 13, 'splitter': 'best', 'max_features': None, 'min_samples_leaf': 1,
     'min_samples_split': 2},
]

In [12]:
train0= pd.read_csv('training data path')
test0 = pd.read_csv('testing data path')

In [13]:
# len(test0)

In [14]:
model = pytorch_prediction(bb0)

In [15]:
ds_names = ['Adult']
regularizeds = [False, True]
dt_best_hyperparams = [
    {'criterion': 'entropy', 'max_depth': 20, 'max_features': 5, 'min_samples_leaf': 40, 'min_samples_split': 50},
    {'criterion': 'gini', 'max_depth': 13, 'splitter': 'best', 'max_features': None, 'min_samples_leaf': 1,
     'min_samples_split': 2},
]
# For each dataset we have
for i, ds_name in enumerate(ds_names):
    # We load the dataset data
    train_set = train0.iloc[:,:39]
    test_set = test0.iloc[:,:39]
    train_label = train0.iloc[:,39:]
    test_label = test0.iloc[:,39:]
    bboxes = []
    # Here we allocate the 6 different bboxes
    for regularized in regularizeds:
        # dt = DecisionTreeBlackBox(db_name=ds_name, regularized=regularized)
        # rf = RandomForestBlackBox(db_name=ds_name, regularized=regularized)
        nn = model
        bboxes.append(nn)
    for bb in tqdm(bboxes):
        # Here we generate the new data according to Trepan
        generator = TrePanGenerator()
        gen = generator.generate(train_set.values, oracle=bb, size=80000)
        data_l = gen[:, -1]
        data = np.delete(gen, -1, axis=1)
        tr_set, ts_set, tr_label, ts_label = train_test_split(data, data_l, stratify=data_l,
                                                              test_size=0.20, random_state=0)
        
        df = pd.DataFrame(data)
        df.to_csv("path to save the data")
        df = pd.DataFrame(data_l)
        df.to_csv("path to save teh labels")
        
        # Best params
        train_explainer(ds_name, dt_best_hyperparams[1], tr_set, tr_label, ts_set, ts_label,
                        ("nn", "regularized"))
        # Regolarized with less depth
        train_explainer_regularized(ds_name, tr_set, tr_label, ts_set, ts_label,
                                        ("nn", "regularized"), max_depth_l=[i for i in range(5, 20)])


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]C:\Users\012709558\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
C:\Users\012709558\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Fitting 3 folds for each of 4320 candidates, totalling 12960 fits


C:\Users\012709558\AppData\Local\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\012709558\AppData\Local\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\012709558\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
C:\Users\012709558\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
 50%|█████████████████████████████████████████                                    

Fitting 3 folds for each of 4320 candidates, totalling 12960 fits


C:\Users\012709558\AppData\Local\anaconda3\lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
C:\Users\012709558\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
C:\Users\012709558\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [36:45<00:00, 1102.57s/it]


In [16]:
# activity